In [28]:
import pandas as pd
import os

In [29]:
DIR = os.getcwd() + "/nfl_predictor"
df = pd.read_csv(DIR + "/nfl_games.csv", index_col=0)

In [30]:
df = df.sort_values("date")
df = df.reset_index(drop=True)

In [31]:
del df["index_opp"]

In [32]:
df

,cmp,att,yds,td,int,sk,yds.1,lng,rate,att.1,...,td.2_max_opp,lng.2_max_opp,fmb_max_opp,fl_max_opp,team_opp,total_opp,home_opp,date,won,season
0,18.0,33.0,194.0,1.0,1.0,3.0,18.0,18.0,69.5,32.0,...,1.0,25.0,1.0,1.0,den,21,1,2016-09-08,False,2016
1,18.0,26.0,178.0,1.0,2.0,2.0,19.0,25.0,69.1,29.0,...,1.0,18.0,0.0,0.0,car,20,0,2016-09-08,True,2016
2,25.0,45.0,227.0,0.0,0.0,0.0,0.0,21.0,69.4,30.0,...,1.0,45.0,1.0,0.0,nyg,20,0,2016-09-11,False,2016
3,20.0,34.0,199.0,2.0,0.0,1.0,0.0,32.0,95.1,25.0,...,1.0,38.0,1.0,0.0,jax,23,1,2016-09-11,True,2016
4,12.0,26.0,190.0,0.0,1.0,3.0,22.0,58.0,55.0,21.0,...,1.0,35.0,2.0,0.0,phi,29,1,2016-09-11,False,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4375,30.0,39.0,241.0,1.0,0.0,2.0,11.0,32.0,100.5,32.0,...,1.0,54.0,1.0,1.0,rav,10,1,2024-01-28,True,2023
4376,20.0,37.0,272.0,1.0,1.0,4.0,17.0,54.0,75.5,16.0,...,1.0,32.0,1.0,0.0,kan,17,0,2024-01-28,False,2023
4377,25.0,41.0,273.0,1.0,0.0,2.0,13.0,25.0,88.8,29.0,...,1.0,51.0,0.0,0.0,sfo,34,1,2024-01-28,False,2023
4378,24.0,39.0,276.0,2.0,0.0,1.0,4.0,45.0,247.6,31.0,...,1.0,52.0,2.0,1.0,kan,25,1,2024-02-11,False,2023


In [33]:
def add_target(team):
    team["target"] = team["won"].shift(-1)
    return team

df = df.groupby("team", group_keys=False).apply(add_target)

/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_24944/1127067056.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("team", group_keys=False).apply(add_target)


In [34]:
df["target"][pd.isnull(df["target"])] = 2

/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_24944/1618397414.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["target"][pd.isnull(df["target"])] = 2
/var/folders/vm/c96v15rs6pv_gl26d74tltsm0000gn/T/ipykernel_24944/161839

In [35]:
df["target"] = df["target"].astype(int,errors="ignore")

In [36]:
df["won"].value_counts()

won
False    2199
True     2181
Name: count, dtype: int64

In [37]:
df["target"].value_counts()

target
0    2183
1    2165
2      32
Name: count, dtype: int64

In [44]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

rr = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(rr, n_features_to_select=18, direction="forward", cv=split)

In [45]:
removed_columns = ["season", "date", "won", "target", "team", "team_opp"]
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [46]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [47]:
sfs.fit(df[selected_columns], df["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=18)

In [48]:
predictors = list(selected_columns[sfs.get_support()])

In [49]:
predictors

['cmp',
 'att',
 'rate',
 'rec',
 'lng.2',
 'att_max',
 'att.1_max',
 'fl_max',
 'total',
 'int_opp',
 'att.1_opp',
 'lng.1_opp',
 'int_max_opp',
 'lng_max_opp',
 'lng.1_max_opp',
 'yds.3_max_opp',
 'td.2_max_opp',
 'lng.2_max_opp']

In [56]:
def backtest(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    seasons = sorted(data["season"].unique())
    
    for i in range(start, len(seasons), step):
        season = seasons[i]
        
        train = data[data["season"] < season]
        test = data[data["season"] == season]
        
        model.fit(train[predictors], train["target"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        
        combined = pd.concat([test["target"], preds], axis=1)
        combined.columns = ["actual", "prediction"]
        
        all_predictions.append(combined)
        
    return pd.concat(all_predictions)

In [57]:
predictions = backtest(df, rr, predictors)

In [54]:
predictions

,actual,predictions
1068,1,0
1069,0,1
1070,1,1
1071,0,0
1072,1,0
...,...,...
4375,1,0
4376,2,0
4377,2,1
4378,2,0


In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(predictions["actual"], predictions["prediction"])

0.5489130434782609

In [ ]:
df.groupby("home").appply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])